# EDA for Walmart Weekly Sale

In [30]:
import pandas as pd
import numpy as np
from numpy import *

import gc
import time
import warnings
import os

In [31]:
data_dir = 'data'
train_file_name = 'train.csv'
features_file_name = 'features.csv'
store_file_name = 'stores.csv'


In [32]:
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [33]:
feature_data = pd.read_csv(os.path.join(data_dir,features_file_name))
feature_data.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [34]:
store_data = pd.read_csv(os.path.join(data_dir,store_file_name))
store_data.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [35]:
store_data.Type.unique()

array(['A', 'B', 'C'], dtype=object)

In [36]:
train_data.Store.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [37]:
# train_data.info()
# train_data['Weekly_Sales'].describe()

# EDA for Store

In [38]:
train_df = train_data.copy()
# train_df.groupby('Store').groups
# # train_df['Store'].value_counts().sort_values(ascending=True, by='Store')
# train_df['Store'].value_counts().sort_values(ascending=True)

In [39]:
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [40]:
# Shift four dats in the future because of a basis of Monday in Forecast in uni of Weekly
train_df.Date = pd.to_datetime(train_df['Date']) + pd.DateOffset(days=3)

# Add foure days
# deeparp_predicts['date'] = pd.to_datetime(deeparp_predicts['date']) + pd.DateOffset(days=4)


In [41]:
train_df[train_df['Store'] == 1].head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-08,24924.50,False
1,1,1,2010-02-15,46039.49,True
2,1,1,2010-02-22,41595.55,False
3,1,1,2010-03-01,19403.54,False
4,1,1,2010-03-08,21827.90,False


In [42]:
train_df = train_df.rename(columns={'Date':'date', 'Store':'item_id'})
train_df.head()

,item_id,Dept,date,Weekly_Sales,IsHoliday
0,1,1,2010-02-08,24924.50,False
1,1,1,2010-02-15,46039.49,True
2,1,1,2010-02-22,41595.55,False
3,1,1,2010-03-01,19403.54,False
4,1,1,2010-03-08,21827.90,False


In [43]:
store_1_2010_02_05_sale = train_df.query('date == "2010-02-05" &  item_id == 1')['Weekly_Sales'].sum()
# assert (store_1_2010_02_05_sale == 1643690.90)

# train_df['Date'] == '2010-02-05' and train_df['Store'] == 1

In [44]:
temp_df = train_df.groupby(['item_id','date'])['Weekly_Sales'].sum()
# df2 = pd.DataFrame(temp_df, columns=['Store','Date','Weekly_Sales'])
stores_sales = pd.DataFrame(temp_df)
stores_sales.head()



Weekly_Sales
item_id date                    
1       2010-02-08    1643690.90
        2010-02-15    1641957.44
        2010-02-22    1611968.17
        2010-03-01    1409727.59
        2010-03-08    1554806.68

In [45]:
# df2.set_index('Date', inplace=True)
stores_sales.reset_index(inplace=True)
# print(stores_sales.groupby('Store').agg({'Date': ['min','max']}))
stores_sales = stores_sales.set_index('date')
stores_sales.head()
print(stores_sales.index.max())
print(stores_sales.index.min())


2012-10-29 00:00:00
2010-02-08 00:00:00


In [46]:

stores_sales.item_id = stores_sales.item_id.astype(str)
stores_sales.dtypes
# df['date'] = pd.to_datetime(df["date"].dt.strftime('%Y-%m'))

# df1.index = pd.to_datetime(df1.index, format = '%m/%d/%Y')
# df1.index = df1.index.strftime('%Y-%m-%d')

# >>> a = datetime.strptime("10/12/2013", "%m/%d/%Y")
stores_sales.index = pd.to_datetime(stores_sales.index, format = '%Y-%m-%d' )
# stores_sales.index = stores_sales.index.strftime("yyyy-MM-dd hh:mm:ss")
stores_sales.index = stores_sales.index.strftime('%Y-%m-%d %H:%M:%S')

#stores_sales.index = pd.to_datetime(stores_sales.index.strftime("yyyy-MM-dd hh:mm:ss"))
cols = ['Weekly_Sales', 'item_id']
stores_sales = stores_sales[cols]

In [47]:
stores_sales.head()

,Weekly_Sales,item_id
2010-02-08 00:00:00,1643690.90,1
2010-02-15 00:00:00,1641957.44,1
2010-02-22 00:00:00,1611968.17,1
2010-03-01 00:00:00,1409727.59,1
2010-03-08 00:00:00,1554806.68,1


## Period of Train and Validation

In [50]:
end_train_date = '2012-07-30' # Non-inclusive
end_val_date = '2012-10-29'

In [51]:

target_stores_sales = stores_sales[stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]

# target_stores_sales = target_stores_sales.['Date' '2011-12-31']
# target_stores_sales

In [52]:
target_stores_sales.tail()

,Weekly_Sales,item_id
2012-06-25 00:00:00,822569.16,45
2012-07-02 00:00:00,773367.71,45
2012-07-09 00:00:00,843361.10,45
2012-07-16 00:00:00,749817.08,45
2012-07-23 00:00:00,737613.65,45


In [53]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [54]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

In [55]:
target_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5805 entries, 2010-02-08 00:00:00 to 2012-07-23 00:00:00
Data columns (total 2 columns):
Weekly_Sales    5805 non-null float64
item_id         5805 non-null object
dtypes: float64(1), object(1)
memory usage: 136.1+ KB


In [56]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 585 entries, 2012-07-30 00:00:00 to 2012-10-22 00:00:00
Data columns (total 2 columns):
Weekly_Sales    585 non-null float64
item_id         585 non-null object
dtypes: float64(1), object(1)
memory usage: 13.7+ KB


In [57]:
validation_stores_sales.head()

,Weekly_Sales,item_id
2012-07-30 00:00:00,1439123.71,1
2012-08-06 00:00:00,1631135.79,1
2012-08-13 00:00:00,1592409.97,1
2012-08-20 00:00:00,1597868.05,1
2012-08-27 00:00:00,1494122.38,1


In [58]:
# # stores_sales.rename(columns={'Date':'date_time', 'Store':'item_ID'})
# # stores_sales = stores_sales.set_index('Date')
# # stores_sales.index

# stores_1_sales = stores_sales[stores_sales['item_id']== "1"]
# stores_1_sales[stores_1_sales.index >= '2011-12-20'].head()

In [59]:

# total_weeks = stores_sales.groupby('Store').count().sum() 
# assert (45 * 143 == int(total_weeks) )
# print(stores_sales.index.min())
# print(stores_sales.index.max())

In [60]:
# Guide: Multiple subplots
# https://jakevdp.github.io/PythonDataScienceHandbook/04.08-multiple-subplots.html

# EDA for Features

In [61]:
feature_df = feature_data.copy()
feature_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [62]:
# Shift four dats in the future because of a basis of Monday in Forecast in uni of Weekly
feature_df.Date = pd.to_datetime(feature_df['Date']) + pd.DateOffset(days=3)


In [63]:
feature_df = feature_df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis=1)

In [64]:
feature_df.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,2010-02-08,42.31,2.572,211.096358,8.106,False
1,1,2010-02-15,38.51,2.548,211.242170,8.106,True
2,1,2010-02-22,39.93,2.514,211.289143,8.106,False
3,1,2010-03-01,46.63,2.561,211.319643,8.106,False
4,1,2010-03-08,46.50,2.625,211.350143,8.106,False


In [65]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 7 columns):
Store           8190 non-null int64
Date            8190 non-null datetime64[ns]
Temperature     8190 non-null float64
Fuel_Price      8190 non-null float64
CPI             7605 non-null float64
Unemployment    7605 non-null float64
IsHoliday       8190 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(4), int64(1)
memory usage: 392.0 KB


In [66]:
# Drop CPI, Unemployment because of missing values
# Drop isHoliday because isHoliday exists in the built-in data of AWS Forecast
feature_df = feature_df.drop(['CPI','Unemployment','IsHoliday'], axis=1)

In [67]:
feature_df.head()

,Store,Date,Temperature,Fuel_Price
0,1,2010-02-08,42.31,2.572
1,1,2010-02-15,38.51,2.548
2,1,2010-02-22,39.93,2.514
3,1,2010-03-01,46.63,2.561
4,1,2010-03-08,46.50,2.625


In [68]:
feature_df = feature_df.rename(columns={'Store':'item_id','Date':'date'})
feature_df.reset_index(inplace=True)
feature_df = feature_df.drop('index', axis=1)
feature_df = feature_df.set_index('date')
feature_df.item_id = feature_df.item_id.astype(str)
feature_df.index = pd.to_datetime(feature_df.index, format = '%Y-%m-%d')
feature_df.index = feature_df.index.strftime('%Y-%m-%d %H:%M:%S')
cols = ['Temperature','Fuel_Price','item_id']
feature_df = feature_df[cols]
feature_df.head()

,Temperature,Fuel_Price,item_id
2010-02-08 00:00:00,42.31,2.572,1
2010-02-15 00:00:00,38.51,2.548,1
2010-02-22 00:00:00,39.93,2.514,1
2010-03-01 00:00:00,46.63,2.561,1
2010-03-08 00:00:00,46.50,2.625,1


In [69]:
feature_df = feature_df[feature_df.index < end_val_date]
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6390 entries, 2010-02-08 00:00:00 to 2012-10-22 00:00:00
Data columns (total 3 columns):
Temperature    6390 non-null float64
Fuel_Price     6390 non-null float64
item_id        6390 non-null object
dtypes: float64(2), object(1)
memory usage: 199.7+ KB


In [70]:
feature_df.isnull().sum()

Temperature    0
Fuel_Price     0
item_id        0
dtype: int64

In [71]:
feature_df.tail()

,Temperature,Fuel_Price,item_id
2012-09-24 00:00:00,65.32,4.038,45
2012-10-01 00:00:00,64.88,3.997,45
2012-10-08 00:00:00,64.89,3.985,45
2012-10-15 00:00:00,54.47,4.000,45
2012-10-22 00:00:00,56.47,3.969,45


In [72]:
feature_df.columns

Index(['Temperature', 'Fuel_Price', 'item_id'], dtype='object')

In [73]:
print(feature_df.index.min())
print(feature_df.index.max())


2010-02-08 00:00:00
2012-10-22 00:00:00


In [74]:
# feature_df.groupby('item_id')['Temperature'].count()

In [75]:


feature_df_25 = feature_df[feature_df['item_id'] == "25"]
feature_df_25.tail()



,Temperature,Fuel_Price,item_id
2012-09-24 00:00:00,57.60,4.038,25
2012-10-01 00:00:00,54.52,3.997,25
2012-10-08 00:00:00,57.58,3.985,25
2012-10-15 00:00:00,43.74,4.000,25
2012-10-22 00:00:00,51.93,3.969,25


In [76]:
stores_sales_25 = target_stores_sales[target_stores_sales['item_id'] == "25"]

In [77]:
pd.merge(stores_sales_25, feature_df_25, 
         left_index=True, right_index=True,
        how='outer')

,Weekly_Sales,item_id_x,Temperature,Fuel_Price,item_id_y
2010-02-08 00:00:00,677231.63,25,21.10,2.784,25
2010-02-15 00:00:00,583364.02,25,19.64,2.773,25
2010-02-22 00:00:00,676260.67,25,24.16,2.745,25
2010-03-01 00:00:00,628516.57,25,29.16,2.754,25
2010-03-08 00:00:00,665750.06,25,29.45,2.777,25
2010-03-15 00:00:00,660619.99,25,40.05,2.818,25
2010-03-22 00:00:00,659795.84,25,45.00,2.844,25
2010-03-29 00:00:00,696687.60,25,45.84,2.854,25
2010-04-05 00:00:00,822486.37,25,47.20,2.850,25
2010-04-12 00:00:00,712647.97,25,61.36,2.869,25


In [79]:
pd.merge(target_stores_sales, feature_df, 
         left_index=True, right_index=True,
        how='outer')

,Weekly_Sales,item_id_x,Temperature,Fuel_Price,item_id_y
2010-02-08 00:00:00,1643690.9,1,42.31,2.572,1
2010-02-08 00:00:00,1643690.9,1,40.19,2.572,2
2010-02-08 00:00:00,1643690.9,1,45.71,2.572,3
2010-02-08 00:00:00,1643690.9,1,43.76,2.598,4
2010-02-08 00:00:00,1643690.9,1,39.70,2.572,5
2010-02-08 00:00:00,1643690.9,1,40.43,2.572,6
2010-02-08 00:00:00,1643690.9,1,10.53,2.580,7
2010-02-08 00:00:00,1643690.9,1,34.14,2.572,8
2010-02-08 00:00:00,1643690.9,1,38.01,2.572,9
2010-02-08 00:00:00,1643690.9,1,54.34,2.962,10


# EDA for Store

In [80]:
store_df = store_data.copy()

In [81]:
store_df = store_df.drop('Size', axis=1)
store_df = store_df.rename(columns={'Store':'item_id', 'Type':'StoreType'})

In [82]:
store_df = store_df.set_index('item_id')

In [83]:
store_df

,StoreType
item_id,
1,A
2,A
3,B
4,A
5,B
6,A
7,B
8,A
9,B
